In [ ]:
%matplotlib inline
import datetime
dateZ0 = datetime.datetime.utcnow()
import matplotlib.pyplot as plt
import numpy as np
from siphon.catalog import get_latest_access_url
import netCDF4
import matplotlib
from mpl_toolkits.basemap import Basemap, cm
from netCDF4 import num2date
import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeat
rtma_catalog="http://thredds.ucar.edu/thredds/catalog/grib/NCEP/RTMA/CONUS_2p5km/catalog.xml"
latest_rtma=get_latest_access_url(rtma_catalog,"OPENDAP")
#print latest_rtma
rtma=netCDF4.Dataset(latest_rtma)
time_var = rtma.variables['time']
timestamp = num2date(time_var[:].squeeze(), time_var.units)
x=(rtma.variables['x'][:])*1000
y=(rtma.variables['y'][:])*1000
T=rtma.variables['Temperature_Analysis_height_above_ground']
Tp=((T[0,0,:,:])*1.8-459.67)
W=rtma.variables['Wind_speed_Analysis_height_above_ground']
Wp=((W[0,0,:,:])*(3600/1609.344))
Td=rtma.variables['Dewpoint_temperature_Analysis_height_above_ground']
Tdp=((Td[0,0,:,:])*1.8-459.67)
C=rtma.variables['Total_cloud_cover_Analysis_entire_atmosphere_single_layer']
Cp=(C[0,:,:])
V=rtma.variables['Visibility_Analysis_surface']
Vp=(V[0,:,:])*(1/1609.344)
Wg=rtma.variables['Wind_speed_gust_Analysis_height_above_ground']
Wgp=((Wg[0,0,:,:])*(3600/1609.344))
state=cfeat.NaturalEarthFeature(category='cultural',
                                name='admin_1_states_provinces_lines',
                                scale='10m', facecolor='none')
lakes=cfeat.NaturalEarthFeature(category='physical',
                                name='lakes',
                                scale='10m', facecolor='none')
grid = rtma[T.grid_mapping]
lon0 = grid.longitude_of_central_meridian
lat0 = grid.latitude_of_projection_origin
lat1 = grid.standard_parallel
earth_radius = grid.earth_radius
globe = ccrs.Globe(ellipse='sphere', semimajor_axis=grid.earth_radius)
crs = ccrs.LambertConformal(central_longitude=lon0, central_latitude=lat0, 
                            standard_parallels=(lat0,lat1), globe=globe)
#print 'READY!'
dateZ1 = datetime.datetime.utcnow()
print dateZ1-dateZ0
print timestamp

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
from siphon.catalog import get_latest_access_url

rtma_catalog = "http://thredds.ucar.edu/thredds/catalog/grib/NCEP/RTMA/CONUS_2p5km/catalog.xml"
latest_rtma_ncss = get_latest_access_url(rtma_catalog, "NetcdfSubset")

# Set up access via NCSS
from siphon.ncss import NCSS
ncss = NCSS(latest_rtma_ncss)

# Create a query to ask for all times in netcdf4 format for
# the Temperature_surface variable, with a bounding box
query = ncss.query()

In [ ]:
query.all_times().accept('netcdf4').variables('Temperature_Analysis_height_above_ground')
query.lonlat_box(north=44, south=36, east=-86, west=-93)

# Get the raw bytes and write to a file.
data = ncss.get_data_raw(query)
with open('test.nc', 'wb') as outf:
    outf.write(data)

In [ ]:
import netCDF4
nc = netCDF4.Dataset('test.nc')

In [ ]:
var='Temperature_Analysis_height_above_ground'
ncvar = nc[var]
ncvar

In [ ]:
grid = nc[ncvar.grid_mapping]
grid

In [ ]:
lon0 = grid.longitude_of_central_meridian
lat0 = grid.latitude_of_projection_origin
lat1 = grid.standard_parallel
earth_radius = grid.earth_radius

In [ ]:
import cartopy
import cartopy.crs as ccrs
#cartopy wants meters, not km
x = (nc['x'][:])*1000
y = (nc['y'][:])*1000
#globe = ccrs.Globe(ellipse='WGS84') #default
globe = ccrs.Globe(ellipse='sphere', semimajor_axis=grid.earth_radius)

crs = ccrs.LambertConformal(central_longitude=lon0, central_latitude=lat0, 
                            standard_parallels=(lat0,lat1), globe=globe)


In [ ]:
import cartopy.feature as cfeat
state=cfeat.NaturalEarthFeature(category='cultural',
                                name='admin_1_states_provinces_lines',
                                scale='10m', facecolor='none')

f=np.arange(20,40,5)
fig = plt.figure(figsize=(14,15))
ax = plt.axes(projection=ccrs.PlateCarree())
ax.set_extent([-92,-87,43, 37],ccrs.Geodetic())
#ax.pcolormesh(x,y,ncvar[0,0].data.squeeze(), transform=crs,zorder=0)
ax.contourf(x,y,ncvar[0,0],f,transform=crs)
ax.coastlines(resolution='10m',color='black')
ax.add_feature(state)
#plt.title(nc[timevar].data[istep]);

In [ ]:
%matplotlib inline
import datetime
dateZ0 = datetime.datetime.utcnow()
dateZ = datetime.datetime.utcnow()
dateZ = datetime.datetime.now()
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap, cm
import netCDF4
from netCDF4 import num2date
import numpy as np
import datetime
import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeat
if dateZ.hour==0:
    URL = 'http://thredds.ucar.edu/thredds/dodsC/grib/NCEP/RTMA/CONUS_2p5km/RTMA_CONUS_2p5km_%04i%02i%02i_%02i%02i.grib2/GC' %\
(dateZ.year,dateZ.month,date.day,(dateZ.hour)+23,0)
    # keep moving back 6 hours until a valid URL found
    validURL = False; ncount = 0
    while (not validURL and ncount < 1):
        print(URL)
        try:
            rtma = netCDF4.Dataset(URL)
            validURL = True
            print 'VALID'
        except RuntimeError:
            date -= datetime.timedelta(hours=1)
            ncount += 1  
            print '***NOT VALID***'
URL = 'http://thredds.ucar.edu/thredds/dodsC/grib/NCEP/RTMA/CONUS_2p5km/RTMA_CONUS_2p5km_%04i%02i%02i_%02i%02i.grib2/GC' %\
(dateZ.year,dateZ.month,dateZ.day,(dateZ.hour)-2,0)
# keep moving back 6 hours until a valid URL found
validURL = False; ncount = 0
while (not validURL and ncount < 1):
    print(URL)
    try:
        rtma = netCDF4.Dataset(URL)
        validURL = True
        print 'VALID'
    except RuntimeError:
        date -= datetime.timedelta(hours=1)
        ncount += 1  
        print '***NOT VALID***'
URL = 'http://thredds.ucar.edu/thredds/dodsC/grib/NCEP/RTMA/CONUS_2p5km/RTMA_CONUS_2p5km_%04i%02i%02i_%02i%02i.grib2/GC' %\
(dateZ.year,dateZ.month,dateZ.day,(dateZ.hour)-1,0)
# keep moving back 6 hours until a valid URL found
validURL = False; ncount = 0
while (not validURL and ncount < 1):
    print(URL)
    try:
        rtma = netCDF4.Dataset(URL)
        validURL = True
        print 'VALID'
    except RuntimeError:
        date -= datetime.timedelta(hours=1)
        ncount += 1  
        print '***NOT VALID***'
URL = 'http://thredds.ucar.edu/thredds/dodsC/grib/NCEP/RTMA/CONUS_2p5km/RTMA_CONUS_2p5km_%04i%02i%02i_%02i%02i.grib2/GC' %\
(dateZ.year,dateZ.month,dateZ.day,(dateZ.hour),0)
# keep moving back 6 hours until a valid URL found
validURL = False; ncount = 0
while (not validURL and ncount < 1):
    print(URL)
    try:
        rtma = netCDF4.Dataset(URL)
        validURL = True
        print 'VALID'
    except RuntimeError:
        date -= datetime.timedelta(hours=1)
        ncount += 1  
        print '***NOT VALID***'
time_var = rtma.variables['time']
timestamp = num2date(time_var[:].squeeze(), time_var.units)
#print 'INT: %s'%timestamp.strftime('%x %H Z')
#print 'INT:%02i%02i'%((dateZ.hour)-1,0)
#timeunits=rtma.variables['time'].units
#print 'INT: %s'%timeunits
x=(rtma.variables['x'][:])*1000
y=(rtma.variables['y'][:])*1000
#x,y=np.meshgrid(x,y)
T=rtma.variables[u'Temperature_Analysis_height_above_ground']
Tp=((T[0,0,:,:])*1.8-459.67)
W=rtma.variables['Wind_speed_Analysis_height_above_ground']
Wp=((W[0,0,:,:])*(3600/1609.344))
Td=rtma.variables['Dewpoint_temperature_Analysis_height_above_ground']
Tdp=((Td[0,0,:,:])*1.8-459.67)
C=rtma.variables['Total_cloud_cover_Analysis_entire_atmosphere_single_layer']
Cp=(C[0,:,:])
V=rtma.variables['Visibility_Analysis_surface']
Vp=(V[0,:,:])*(1/1609.344)
Wg=rtma.variables['Wind_speed_gust_Analysis_height_above_ground']
Wgp=((Wg[0,0,:,:])*(3600/1609.344))

state=cfeat.NaturalEarthFeature(category='cultural',
                                name='admin_1_states_provinces_lines',
                                scale='10m', facecolor='none')
lakes=cfeat.NaturalEarthFeature(category='physical',
                                name='lakes',
                                scale='10m', facecolor='none')
grid = rtma[T.grid_mapping]
lon0 = grid.longitude_of_central_meridian
lat0 = grid.latitude_of_projection_origin
lat1 = grid.standard_parallel
earth_radius = grid.earth_radius
globe = ccrs.Globe(ellipse='sphere', semimajor_axis=grid.earth_radius)
crs = ccrs.LambertConformal(central_longitude=lon0, central_latitude=lat0, 
                            standard_parallels=(lat0,lat1), globe=globe)
dateZ1 = datetime.datetime.utcnow()
print dateZ1-dateZ0

In [ ]:
a=92-87
b=43-37
print 'figsize=%s,%s'%(a,b)
c=abs(a-b)
print 'Diff=%s'%c

In [ ]:
from mpl_toolkits.basemap import Basemap
m = Basemap(width=12000000,height=9000000,rsphere=6371200.0,
            projection='lcc',resolution='c',lat_1=25.,lat_2=25,
            lat_0=25,lon_0=265.)
m.fillcontinents(color='coral',lake_color='aqua')
lon, lat = -88.832842, 42.275721 # Location of Boulder
# convert to map projection coords.
# Note that lon,lat can be scalars, lists or numpy arrays.
xpt,ypt = m(lon,lat)
# convert back to lat/lon
lonpt, latpt = m(xpt,ypt,inverse=True)
m.plot(xpt,ypt,'bo')  # plot a blue dot there
# put some text next to the dot, offset a little bit
# (the offset is in map projection coordinates)
plt.text(xpt+100000,ypt+100000,'Belvidere (%5.1fW,%3.1fN)' % (lonpt,latpt))

In [ ]:
print 'xpt=%s'%xpt
print 'ypt=%s'%ypt

In [ ]:
from scipy.ndimage.filters import gaussian_filter
from matplotlib.pyplot import contour, show
dateZ = datetime.datetime.utcnow()
sigma=1 
data=gaussian_filter(Tp,sigma)
c = np.arange(30,70,5)
fig=plt.figure(figsize=(14,15)) #14,15
#plt.figure()
ax=plt.axes(projection=ccrs.PlateCarree())
ax.set_extent([-92,-87,43, 37],ccrs.Geodetic())
a=ax.pcolormesh(x,y,Tp.squeeze(),transform=crs,zorder=0,
                cmap='nipy_spectral')
#a=ax.contourf(x,y,Tp.squeeze(),c,transform=crs,zorder=0,
#                cmap='nipy_spectral')
#ax.contour(x,y,Tp.squeeze(),c,transform=crs,colors='k')
t=ax.contour(x,y,data.squeeze(),c,transform=crs,colors='k')
ax.add_feature(state)
ax.add_feature(lakes,edgecolors='blue')
ax.set_title(u'RTMA T°F Analysis @ %s'%timestamp.strftime('%x %H Z'),
          fontsize=20)
colorbar_ax = fig.add_axes([.86,0.125,0.05,0.775])#[0.7, 0.1, 0.05, 0.8]
plt.colorbar(a,cax=colorbar_ax) #a
g=ax.text(0.331,0.0045,dateZ.strftime('Generated @ %x %H:%M:%S Z'),
        horizontalalignment='right',verticalalignment='bottom',
        transform=ax.transAxes,fontsize=12,color='w',
        fontweight='semibold')
g.set_bbox(dict(color='black',alpha=0.65))
#plt.clabel(t, fontsize=10)

In [ ]:
fig=plt.figure(figsize=(14,15)) 
ax=plt.axes(projection=ccrs.PlateCarree())
ax.set_extent([-92,-87,43,37],ccrs.Geodetic())
b=ax.pcolormesh(x,y,Tdp.squeeze(),cmap='nipy_spectral',transform=crs,zorder=0)
ax.add_feature(state)
ax.add_feature(lakes,edgecolors='blue')
ax.set_title(u'RTMA Td°F Analysis @ %s'%timestamp.strftime('%x %H Z'),
          fontsize=20)
colorbar_ax = fig.add_axes([.86, 0.125, 0.05, .775])#[0.7, 0.1, 0.05, 0.8]
plt.colorbar(b, cax=colorbar_ax)

In [ ]:
fig=plt.figure(figsize=(14,15)) 
ax=plt.axes(projection=ccrs.PlateCarree())
ax.set_extent([-92,-87,43,37],ccrs.Geodetic())
c=ax.pcolormesh(x,y,Wp.squeeze(),cmap='nipy_spectral',transform=crs,zorder=0)
#ax.pcolormesh(x,y,Wp.squeeze(),transform=crs,zorder=0)
ax.add_feature(state)
ax.add_feature(lakes,edgecolors='blue')
ax.set_title(u'RTMA Wind (mph) Analysis @ %s'%timestamp.strftime('%x %H Z'),
          fontsize=20)
colorbar_ax=fig.add_axes([.86,0.125,0.05,0.775])#[0.7, 0.1, 0.05, 0.8]
plt.colorbar(c,cax=colorbar_ax)

In [ ]:
fig=plt.figure(figsize=(14,15)) 
ax=plt.axes(projection=ccrs.PlateCarree())
ax.set_extent([-92,-87,43,37],ccrs.Geodetic())
d=ax.pcolormesh(x,y,Cp.squeeze(),cmap='magma_r',transform=crs,zorder=0)
ax.add_feature(state,edgecolors='red')
ax.add_feature(lakes,edgecolors='blue')
ax.set_title(u'RTMA Cloud Cover (%%) Analysis @ %s'%timestamp.strftime('%x %H Z'),
          fontsize=20)
colorbar_ax=fig.add_axes([0.86,0.125,0.05,0.775])#[0.7, 0.1, 0.05, 0.8]
plt.colorbar(d,cax=colorbar_ax)

In [ ]:
#cf=np.arange(0,13,1)
#fig=plt.figure() 
fig=plt.figure(figsize=(14,15))
ax=plt.axes(projection=ccrs.PlateCarree())
ax.set_extent([-92,-87,43,37],ccrs.Geodetic())
s=ax.pcolormesh(x,y,Vp.squeeze(),cmap='Set1',transform=crs,zorder=0)
#g=ax.contourf(x,y,Vp.squeeze(),cf,cmap='Set1')
ax.add_feature(state)
ax.add_feature(lakes,edgecolors='blue')
ax.set_title(u'RTMA Visibility (mi) Analysis @ %s'%timestamp.strftime('%x %H Z'),
          fontsize=20)
colorbar_ax = fig.add_axes([.86, 0.125, 0.05, .775])#[0.7, 0.1, 0.05, 0.8]
plt.colorbar(s, cax=colorbar_ax)
#plt.colorbar(g, cax=colorbar_ax)

In [ ]:
fig=plt.figure(figsize=(14,15)) 
ax=plt.axes(projection=ccrs.PlateCarree())
ax.set_extent([-92,-87,43,37],ccrs.Geodetic())
e=ax.pcolormesh(x,y,Wgp.squeeze(),cmap='nipy_spectral',transform=crs,zorder=0)
ax.add_feature(state)
ax.add_feature(lakes,edgecolors='blue')
ax.set_title(u'RTMA Wind Gust (mph) Analysis @ %s'%timestamp.strftime('%x %H Z'),
          fontsize=20)
colorbar_ax=fig.add_axes([0.86,0.125,0.05,0.775])#[0.7, 0.1, 0.05, 0.8]
plt.colorbar(e,cax=colorbar_ax)

In [ ]:
# x=rtma.variables['x'][:]
y=rtma.variables['y'][:]
x,y=np.meshgrid(x,y)
T=rtma.variables[u'Temperature_Analysis_height_above_ground']
Tp=((T[0,0,:,:])*1.8-459.67)

In [ ]:
# -*- coding: utf-8 -*-
fig=plt.figure(figsize=(12,8)) #x:1200,y:800
cf=np.arange(20,80,1)
c=np.arange(20,80,5)
plt.contourf(x,y,Tp,cf,cmap='nipy_spectral')
plt.contour(x,y,Tp,c,colors='black')
plt.title(u'RTMA T°F Analysis @ %s'%timestamp.strftime('%x %H Z'),
          fontsize=20)
plt.xlim(-400,800)
plt.ylim(1400,2200)

In [ ]:
W=rtma.variables['Wind_speed_Analysis_height_above_ground']
Wp=((W[0,0,:,:])*(3600/1609.344))

In [ ]:
# -*- coding: utf-8 -*-
fig=plt.figure(figsize=(12,8))
c1=np.arange(0,50,.5)
plt.contourf(x,y,Wp,c1,cmap='nipy_spectral')
plt.title(u'RTMA Wind (mph) Analysis @ %s'%timestamp.strftime('%x %H Z'),
          fontsize=20)
plt.xlim(-400,800)
plt.ylim(1400,2200)

In [ ]:
print 'Wmin %s mph'%np.amin(Wp)
print 'Wmax %s mph'%np.amax(Wp)

In [ ]:
Td=rtma.variables['Dewpoint_temperature_Analysis_height_above_ground']
Tdp=((Td[0,0,:,:])*1.8-459.67)

In [ ]:
fig=plt.figure(figsize=(12,8))
plt.xlim(-400,800)
plt.ylim(1400,2200)
c2=np.arange(32,70,1)
plt.contourf(x,y,Tdp,c2,cmap='nipy_spectral')
plt.title(u'RTMA Td°F Analysis @ %s'%timestamp.strftime('%x %H Z'),
          fontsize=20)

In [ ]:
print 'Tdmin %s mph'%np.amin(Tdp)
print 'Tdmax %s mph'%np.amax(Tdp)